# Step 1 - Subset & Ressampling & Mask w/ IdePix

In [1]:
import os
import subprocess

# Here we define the input and output folders and the file name pattern for the input and output files.
 
output_folder = 'l1c_ressampled_masked/'
name_start = 'S2_MSIL1C_'
name_end = '_samp_mask.dim'
input_folder = 'L1C_zip_files/'

def process_zip_files(output_folder, name_start, name_end, input_folder):
    """
    Process the zip files in the input folder by resampling and masking them using the gpt tool.
    
    This function iterates over the zip files in the input folder, extracts the acquisition date from the file name,
    and generates the output file path. It then executes the gpt command to subset, resample and mask the input file.

    """

    for zip_file in os.listdir(input_folder):
        if zip_file.endswith('.zip'):
            product_basename = os.path.basename(zip_file)
            ac_date = product_basename.split('_')[2]
            output_pathname = os.path.join(output_folder, name_start + ac_date + name_end)
            input_pathname = os.path.join(input_folder, zip_file)

            cmd = [ 
                'gpt',
                'data/STEP1_subset_resample_mask.xml',
                '-PInput=' + input_pathname,
                '-POutput=' + output_pathname
            ]

            subprocess.run(cmd)

# Here we call the function to process the zip files in the input folder.
process_zip_files(output_folder, name_start, name_end, input_folder)

# Step 2 - Application of C2RCC w/ Meteo Data

In [1]:
import os
import subprocess

# Here we define the input and output folders and the file name pattern for the input and output files.

output_folder = 'c2rcc/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP2_c2rcc_correction.xml'
input_folder = 'l1c_ressampled_masked/'

def process_c2rcc_correction(input_folder, output_folder, met_data, param_file):
    """
    Process C2RCC correction on the input files in the specified folder.

    Args:
        input_folder (str): Path to the folder containing the input files.
        output_folder (str): Path to the folder where the output files will be saved.
        met_data (str): Path to the meteorological data file.
        param_file (str): Path to the C2RCC correction parameter file.

    Returns:
        None
    """
    old_end = '.dim'
    new_end = '_C2RCC.dim'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)
            idate = name.split('_')[2].split('T')[0]

            """
            We read the meteorological data for the acquisition date of the current file and 
            extract the ozone, pressure and temperature values.
            """

            with open(met_data, 'r') as met_file:
                for line in met_file:
                    if idate in line:
                        parts = line.split(',')
                        temp = parts[1]
                        press = parts[2]
                        ozone = parts[3]
                        ozone = ozone.replace('\n', '')
                        break

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                'c2rcc.msi',
                '-SsourceProduct=' + dim_file_path,
                '-p', param_file,
                '-Pozone=' + ozone,
                '-Ppress=' + press,
                '-Ptemperature=' + temp,
                '-t', output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Running the function
process_c2rcc_correction(input_folder, output_folder, met_data, param_file)

STDOUT: Executing operator...
10%20%...30%...42%...54%...64%...76%...88%...100%102% done.
Writing...
....10%....20%...30%...42%...54%...64%...76%...88%...100% done.

STDERR: INFO: org.esa.snap.python.gpf.PyOperatorSpi: Python operator 'py_sambuca_snap_op' registered (Python module: 'sambuca_snap_op', class: 'sambuca_snap_op', root: 'C:\Users\arthu\AppData\Roaming\SNAP\modules\org-esa-sen2coral-sen2coral-inversion.jar')
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.s3tbx.c2rcc.ancillary.AtmosphericAuxdataBuilder: Atmospheric auxdata product can't be used. At least one is not specified. Using constant values for ozone (279.259583) and surface pressure (1002.9475).
INFO: org.esa.snap.core.gpf.common.WriteOp: Start writing product c2rcc.msi

## Step 2.1 - Application of C2RCC w/ Meteo Data (excluding Ozone data)

In [ ]:
import os
import subprocess

output_folder = 'c2rcc_without_ozone/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP2_c2rcc_correction.xml'
input_folder = 'l1c_ressampled_masked/'

def process_c2rcc_correction_without_ozone(input_folder, output_folder, met_data, param_file):

    
    """
    Process C2RCC correction on the input files in the specified folder.

    Args:
        input_folder (str): Path to the folder containing the input files.
        output_folder (str): Path to the folder where the output files will be saved.
        met_data (str): Path to the meteorological data file.
        param_file (str): Path to the C2RCC correction parameter file.

    Returns:
        None
    """

    old_end = '.dim'
    new_end = '_C2RCC.dim'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)
            idate = name.split('_')[2].split('T')[0]

            """
            We read the meteorological data for the acquisition date of the current file and 
            extract the pressure and temperature values.
            """

            with open(met_data, 'r') as met_file:
                for line in met_file:
                    if idate in line:
                        parts = line.split(',')
                        temp = parts[1]
                        press = parts[2]
                        break

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                'c2rcc.msi',
                '-SsourceProduct=' + dim_file_path,
                '-p', param_file,
                '-Ppress=' + press,
                '-Ptemperature=' + temp,
                '-t', output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Running the function
process_c2rcc_correction_without_ozone(input_folder, output_folder, met_data, param_file)

## Step 2.2 - Application of C2RCC w/ Meteo Data + C2-NETS

In [ ]:
import os
import subprocess

output_folder = 'c2rcc_complex_nets/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP22_c2rcc_complex_nets_correction.xml'
old_end = '.dim'
new_end = '_C2RCC.dim'

input_folder = 'l1c_ressampled_masked/'

process_c2rcc_correction(input_folder, output_folder, met_data, param_file)


# Step 3 - Bio-optical models calculation

In [2]:
"""
This script performs 6 bands manipulations of Rrs (Remote Sensing Reflectance) to estimate chlorophyll-a concentration.
It applies different equations to calculate the chlorophyll-a concentration using the Rrs values of different bands.

The following equations are used to calculate the chlorophyll-a concentration using the Rrs values of different bands:

1. 3_bands_model_chl_conc - 161.24*((1/ rrs_B4 - 1/ rrs_B5)/(1/ rrs_B6 - 1/ rrs_B5))+28.04
2. MCI - rrs_B5 - rrs_B4 - (rrs_B6 -rrs_B4)*((709-665)/(754-665))
3. empirical_2_bands_equation1 - 136.3 * (rrs_B6/rrs_B4) - 16.2
4. empirical_2_bands_equation2 - 25.28*((rrs_B5/rrs_B4)*(rrs_B5/rrs_B4)) + 14.85 * (rrs_B5/rrs_B4) - 15.18
5. empirical_3_bands_equation1 - 117.42 * ((1/rrs_B4 - 1/rrs_B5))*rrs_B6 + 23.174
6. empirical_3_bands_equation3 - 315.5 * (((1/rrs_B4 - 1/rrs_B5))*rrs_B6)*(((1/rrs_B4 - 1/rrs_B5))*rrs_B6)+ 215.95 * ((1/rrs_B4 - 1/rrs_B5))*rrs_B6 + 25.66

The script takes input files from the 'c2rcc' folder, applies the calculations defined in the 'STEP3_chla_algorithms_calculation.xml' graph file,
and saves the output files with modified names in the 'c2rcc_final_bands' folder.

The script uses the 'gpt' command-line tool to execute the graph file and perform the calculations.

Note: The 'gpt' command-line tool must be installed and accessible in the system's PATH for the script to work properly.
"""

import os
import subprocess

output_folder = 'c2rcc_final_bands/'
input_folder = 'c2rcc/'

graph_file = 'data/STEP3_chla_algorithms_calculation.xml'

old_end = '.dim'
new_end = '_mathbands.dim'

for dim_file in os.listdir(input_folder):
    if dim_file.endswith('.dim'):
        dim_file_path = os.path.join(input_folder, dim_file)
        name = os.path.basename(dim_file_path)

        output_name = name.replace(old_end, new_end)
        output_path = os.path.join(output_folder, output_name)

        cmd = [ 
            'gpt',
            graph_file,
            '-PInput=' + dim_file_path,
            '-POutput=' + output_path
        ]

        result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("STDOUT:", result.stdout)
        print("STDERR:", result.stderr)


STDOUT: Executing processing graph
.11%.22%.33%.45%.56%.67%.78%...89% done.

STDERR: INFO: org.esa.snap.python.gpf.PyOperatorSpi: Python operator 'py_sambuca_snap_op' registered (Python module: 'sambuca_snap_op', class: 'sambuca_snap_op', root: 'C:\Users\arthu\AppData\Roaming\SNAP\modules\org-esa-sen2coral-sen2coral-inversion.jar')
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.hsqldb.persist.Logger: dataFileCache open start

STDOUT: Executing processing graph
.11%.22%.33%.45%.56%.67%.78%...89% done.

STDERR: INFO: org.esa.snap.python.gpf.PyOperatorSpi: Python operator 'py_sambuca_snap_op' registered (Python module: 'sambuca_snap_op', class: 'sambuca_snap_op', root: 'C:\Users\arthu\AppData\Roaming\SNAP\modules\org-esa-sen2coral-sen2coral-inversion.jar')
INFO: org.esa.snap.core.gpf.operators.toolada

# Step 4 - Extract values of the bands

### Results of C2RCC with Ozone Data - window 3x3

In [23]:
import os
import subprocess

output_folder = 'stats_results_windows'
param_file = 'data/STEP4_1_pixel_values_extraction_3x3.xml'

input_folder = 'c2rcc_final_bands/*.dim'

cmd = [
    'gpt',
    param_file,
    '-PInput=' + input_folder,
    '-POutput='+ output_folder
]

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

### Results of C2RCC with Ozone Data - window 5x5

In [26]:
import os
import subprocess

output_folder = 'stats_results_windows'
param_file = 'data/STEP4_2_pixel_values_extraction_5x5.xml'

input_folder = 'c2rcc_final_bands/*.dim'

cmd = [
    'gpt',
    param_file,
    '-PInput=' + input_folder,
    '-POutput='+ output_folder
]

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

### Consolidation of window results

In [ ]:
import pandas as pd

# Read the csv files and skipping the first 6 rows 
df_3x3 = pd.read_csv('stats_results_window_3x3/pixEx_3x3_BandMath_measurements.txt', delimiter='\t', skiprows=6)
df_5x5 = pd.read_csv('stats_results_window_3x3/pixEx_5x5_BandMath_measurements.txt', delimiter='\t', skiprows=6)

# Selecting the columns that we want to analyse

columns = ['Date(yyyy-MM-dd)', '3_bands_model_chl_conc_mean', '3_bands_model_chl_conc_sigma',
           '3_bands_model_chl_conc_num_pixels', 'kd_z90max_mean', 'kd_z90max_sigma', 'kd_z90max_num_pixels',
           'conc_tsm_mean', 'conc_tsm_sigma', 'conc_tsm_num_pixels', 'conc_chl_mean', 'conc_chl_sigma',
           'conc_chl_num_pixels', 'MCI_mean', 'MCI_sigma', 'MCI_num_pixels', 'empirical_2_bands_equation1_mean',
           'empirical_2_bands_equation1_sigma', 'empirical_2_bands_equation1_num_pixels',
           'empirical_2_bands_equation2_mean', 'empirical_2_bands_equation2_sigma',
           'empirical_2_bands_equation2_num_pixels', 'empirical_3_bands_equation1_mean',
           'empirical_3_bands_equation1_sigma', 'empirical_3_bands_equation1_num_pixels',
           'empirical_3_bands_equation3_mean', 'empirical_3_bands_equation3_sigma',
           'empirical_3_bands_equation3_num_pixels']

# Creating a new column with the window size of each row
df_3x3 = df_3x3[columns]
df_3x3['window_size'] = 3
df_5x5 = df_5x5[columns]
df_5x5['window_size'] = 5

# Display DataFrame
display(df_3x3)
display(df_5x5)

# Concatenate the two DataFrames and save the results to a csv file
df_windows = pd.concat([df_3x3, df_5x5], ignore_index=True)
display(df_windows)

df_windows.to_csv('data/results_window_calculation.csv', index=False)

### Results of C2RCC with Ozone Data - All parameters

In [8]:
import os
import subprocess

output_folder = 'stats_results_final_bands/'
param_file = 'data/STEP4_3_all_pixels_extraction_StatisticOperator.xml'
input_folder = 'c2rcc_final_bands/'

def process_files(output_folder, param_file, input_folder):
    """
    Process the files in the input folder by running a command line tool that extract 
    the pixel values of bands selected in the parameter file.
    
    The bands selected are: conc_chl, conc_tsm, 3_bands_model_chl_conc, MCI, empirical_2_bands_equation1, 
    empirical_2_bands_equation2, empirical_3_bands_equation1, empirical_3_bands_equation3.

    The output files will be .txt and saved in the output folder.
    """

    old_end = '.dim'
    new_end = '_stats.txt'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                param_file,
                '-PInput=' + dim_file_path,
                '-POutput='+ output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

process_files(output_folder, param_file, input_folder)


In [12]:
import pandas as pd
import os

def process_data(input_folder):
    """
    Process the data files in the given input folder.
    
    Args:
        input_folder (str): Path to the folder containing the data files.
        
    Returns:
        pd.DataFrame: Consolidated dataframe containing the processed data.
    """
    # Create an empty list to store the dataframes
    dfs = []

    # Iterate over the files in the input folder
    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('_stats.txt'):
            # Extract the date from the file name
            name = os.path.basename(dim_file)
            idate = name.split('_')[2].split('T')[0]

            # Read the data from the file into a dataframe
            df_virtual = pd.read_csv(os.path.join(input_folder, dim_file), sep='\t')
            df_virtual['date'] = pd.to_datetime(idate, format='%Y%m%d')

            # Append the dataframe to the list
            dfs.append(df_virtual)

    # Concatenate all the dataframes into a single dataframe
    consolidated_df = pd.concat(dfs, ignore_index=True)

    return consolidated_df

# Running the function
input_folder = 'stats_results_final_bands/'
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv('data/results_final_bands.csv', index=False)


,name,Band,average,max_error,maximum,median,minimum,p80_threshold,p85_threshold,p90_threshold,sigma,total,date
0,world,3_bands_model_chl_conc,12.8323,0.0231,26.5971,12.8952,3.4911,14.9517,15.4138,16.0839,2.9408,2559,2022-10-07
1,world,MCI,0.0000,0.0000,0.0019,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0001,34254,2022-10-07
2,world,conc_chl,14.9729,0.0346,34.6434,14.3088,0.0019,18.5351,20.3711,25.0823,6.7115,2559,2022-10-07
3,world,conc_tsm,2.8690,0.0158,15.8134,2.4614,0.0122,4.4839,4.9422,5.4952,1.9014,2559,2022-10-07
4,world,empirical_2_bands_equation1,21.6171,0.0408,41.7606,21.9678,0.9507,25.0286,25.7631,26.9874,5.0381,2559,2022-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,world,conc_tsm,0.9802,0.0147,15.1318,0.8526,0.4262,1.1467,1.2203,1.2938,0.5412,1886,2023-09-25
212,world,empirical_2_bands_equation1,17.2390,0.0260,32.9732,17.1304,6.9588,20.3822,21.0066,21.7610,3.5927,1886,2023-09-25
213,world,empirical_2_bands_equation2,11.3086,0.0151,19.4234,11.3588,4.3212,13.2164,13.5637,13.8809,2.2295,1886,2023-09-25
214,world,empirical_3_bands_equation1,14.6966,0.0080,19.5269,14.7092,11.5768,15.5996,15.7506,15.9096,1.0854,1886,2023-09-25


### Results of C2RCC without Ozone data

In [ ]:
output_folder = 'stats_results_without_ozone/'
param_file = 'data/[OLD]STEP4_chl_tsm_extraction_all_pixels.xml'
input_folder = 'c2rcc_without_ozone/'

# Call the function to start the processing
process_files(input_folder, output_folder, param_file)

# Save the results to a csv file
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv(input_folder + 'results_without_ozone.csv', index=False)


### Results of Complex Waters Algorithm

In [ ]:
output_folder = 'stats_results_complex_nets/'
param_file = 'data/[OLD]STEP4_chl_tsm_extraction_all_pixels.xml'
input_folder = 'c2rcc_complex_nets/'

# Call the function to start the processing of the statistics extraction files
process_files(input_folder, output_folder, param_file)


# Save the results to a csv file
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv(input_folder + 'results_complex_nets.csv', index=False)
